# ONNX Documentation

The ONNX (Open Neural Network eXchange) model format is an open standard for representing machine learning algorithms as such it can be used for creating your model in any of the Machine Learning frameworks that have the ability to be converted to the ONNX format. Some of the available frameworks are: Tensorflow, Keras, PyTorch, scikit-learn, Apple Core ML, Spark ML, LightGBM, libsvm, XGBoost, among others.

Additional Resources:
- https://onnx.ai/supported-tools.html
- https://github.com/onnx/onnx/blob/main/docs/PythonAPIOverview.md

**Table of Contents**

* [Onnx Conversion](#1)
    * [Different formats to ONNX](#1-1)
        * [TensorFlow/Keras](#1-1-1)
        * [PyTorch](#1-1-2)
        * [Scikit-learn](#1-1-3)
        * [XGBoost](#1-1-4)
    * [Conversion Pitfalls](#1-2)
* [Graph Modifications](#2)
    * [Onnxruntime optimization](#2-0)
    * [Remapping model inputs](#2-1)
    * [Renaming model outputs](#2-2)
    * [Merging graphs](#2-3)
* [Time Series Data](#3)
    * [Kshape](#3-1)
    * [DTW](#3-2)
    * [Resampling](#3-3)
* [ONNX Prediction](#4)

## Onnx Conversion <a class="anchor" id="1"></a>


### Different formats to ONNX <a class="anchor" id="1-1"></a>

#### TensorFlow/Keras to ONNX <a class="anchor" id="1-1-1"></a>

The simplest way to convert a TensorFlow or a Keras model to ONNX is to  save the model in tensorflow's SavedModel format, as stated in the documentation (https://www.tensorflow.org/api_docs/python/tf/keras/models/save_model)

Additional Resource:
- https://github.com/onnx/tensorflow-onnx
- https://www.tensorflow.org/guide/saved_model
- https://www.tensorflow.org/tutorials/keras/save_and_load#savedmodel_format

In [ ]:
import tensorflow as tf
from acslibrary.onnx_utils.conversion import tensorflow_to_onnx

# Example keras model
tf_model = tf.keras.Sequential(
  [
    tf.keras.layers.Dense(5, input_shape=(3,)),
    tf.keras.layers.Softmax(),
  ]
)

# Save model in onnx format
onnx_model_path = './models/tf_model.onnx'
tensorflow_to_onnx(tf_model, save_onnx_filename=onnx_model_path)

#### PyTorch to ONNX <a class="anchor" id="1-1-2"></a>

PyTorch has native support (https://pytorch.org/docs/stable/onnx.html) for ONNX export and as such can be exported directly from code.

Additional Resources:
- https://pytorch.org/tutorials/advanced/super_resolution_with_onnxruntime.html

In [ ]:
import torch
from acslibrary.onnx_utils.conversion import pytorch_to_onnx

# Define the PyTorch model to export
class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.linear = torch.nn.Linear(10, 5)
        self.relu = torch.nn.ReLU()
    
    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        return x

# Create an instance of the model
torch_model = MyModel()

# Define an example input to use for tracing the model
example_input = torch.randn(1, 10)

# Export the model to ONNX format
save_onnx_filename = "./models/pytorch_model.onnx"
pytorch_to_onnx(torch_model,
                example_input,
                save_onnx_filename,
                export_params=True,             # store the trained parameter weights inside the model file
                input_names = ['input'],        # the model's input names
                output_names = ['output'],      # the model's output names
                dynamic_axes={
                    'input' : {0 : 'batch_size'},    # variable length axes
                    'output' : {0 : 'batch_size'},
                })

#### SciKit-Learn to ONNX <a class="anchor" id="1-1-3"></a>

`sklearn_to_onnx` converts either an sklearn model or Pipeline to ONNX format, allowing to define initial_types or infer them from train data X. The function saves the model to *save_onnx_filename* if the parameter is declared, otherwise it returns the converted model.

Additional Resources:
- http://onnx.ai/sklearn-onnx/
- https://github.com/onnx/sklearn-onnx

In [1]:
from skl2onnx.common.data_types import FloatTensorType

from acslibrary.onnx_utils.conversion import sklearn_to_onnx

# Example classification model (needs X_train and y_train)
from sklearn.tree import DecisionTreeClassifier
X_train = [[0, 0], [1, 1]]
y_train = [0, 1]
clf = DecisionTreeClassifier().fit(X_train, y_train)

# Define the input type and shape
initial_type = [('float_input', FloatTensorType([None, 2]))]

# Convert model to onnx
save_onnx_filename = "./models/sklearn_tree_model.onnx"
sklearn_to_onnx(clf, initial_types=initial_type, save_onnx_filename=save_onnx_filename)

2023-04-25 14:08:25.452431: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ir_version: 8
producer_name: "skl2onnx"
producer_version: "1.14.0"
domain: "ai.onnx"
model_version: 0
doc_string: ""
graph {
  node {
    input: "float_input"
    output: "label"
    output: "probabilities"
    name: "TreeEnsembleClassifier"
    op_type: "TreeEnsembleClassifier"
    attribute {
      name: "class_ids"
      ints: 0
      ints: 0
      type: INTS
    }
    attribute {
      name: "class_nodeids"
      ints: 1
      ints: 2
      type: INTS
    }
    attribute {
      name: "class_treeids"
      ints: 0
      ints: 0
      type: INTS
    }
    attribute {
      name: "class_weights"
      floats: 0.0
      floats: 1.0
      type: FLOATS
    }
    attribute {
      name: "classlabels_int64s"
      ints: 0
      ints: 1
      type: INTS
    }
    attribute {
      name: "nodes_falsenodeids"
      ints: 2
      ints: 0
      ints: 0
      type: INTS
    }
    attribute {
      name: "nodes_featureids"
      ints: 0
      ints: 0
      ints: 0
      type: INTS
    }
    attr

#### XGBoost to ONNX <a class="anchor" id="1-1-4"></a>

XGBoost is an optimized distributed gradient boosting library designed to be highly efficient, flexible and portable. It implements machine learning algorithms under the Gradient Boosting framework.

Additional Resources:
- https://xgboost.readthedocs.io/en/stable/index.html
- https://onnx.ai/sklearn-onnx/auto_tutorial/plot_gexternal_xgboost.html

In [ ]:
import numpy as np
from sklearn.pipeline import Pipeline

from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from skl2onnx.common.data_types import FloatTensorType

from xgboost import XGBClassifier
import onnxruntime as ort

from acslibrary.onnx_utils.conversion import xgboost_to_onnx

data = load_iris()
X = data.data[:, :2]
y = data.target

ind = np.arange(X.shape[0])
np.random.shuffle(ind)
X = X[ind, :].copy()
y = y[ind].copy()

pipe = Pipeline([('scaler', StandardScaler()),
                 ('xgb', XGBClassifier(n_estimators=3))])
pipe.fit(X, y)

save_onnx_filename = "./models/pipeline_xgboost_classifier.onnx"
initial_type = [('input', FloatTensorType([None, 2]))]
xgboost_to_onnx(pipe, xgb_mode="classifier", save_onnx_filename=save_onnx_filename, initial_type=initial_type)

# Compare the predictions
print("\n Predictions with XGBoost")
print("predict", pipe.predict(X[:5]))
print("predict_probabilities", pipe.predict_proba(X[:1]))

print("\n Predictions with onnxruntime")
sess = ort.InferenceSession(save_onnx_filename)
pred_onx = sess.run(None, {"input": X[:5].astype(np.float32)})
print("predict", pred_onx[0])
print("predict_probabilities", pred_onx[1][:1])

### Conversion Pitfalls <a class="anchor" id="1-2"></a>

#### <ins>Regex tokenization (ONNX uses different regex engine)</ins>
When converting a model to ONNX format, the model is represented in a graph format that describes the computations in the model. The ONNX runtime engine uses this graph to execute the model on different hardware platforms. However, the ONNX runtime engine only supports a subset of regular expressions, and some regex expressions can cause errors during runtime.

Tokenization is a process of splitting a piece of text into individual tokens, such as words or phrases. Regex tokenization is a popular method of tokenization because it can handle complex patterns in the text. However, when converting a python-based Tokenizer (i.e. scikit-learn or pyspark), the ONNX runtime engine may not be able to handle some of the regex expressions. This can cause the model to fail during runtime, and the output may be different from the expected results.

To avoid this problem, it is recommended to use a tokenization method that is supported by the ONNX runtime engine, for this please use the [ONNX Tokenizer](http://www.xavierdupre.fr/app/mlprodict/helpsphinx/onnxops/onnx_commicrosoft_Tokenizer.html#id1) by passing the regex expression to `tokenexp` attribute until it matches the expected output.



#### <ins>Unsupported operators and float32/float64 verification</ins>

The ONNX format does not support all the operators that are available in every deep learning framework. When converting a model, it is important to check whether all the operators used in the model are supported by ONNX.

Even when the operators are available in the ONNX format, it can happen that some are not able to do float64. ONNX handles these cases by adding constants or casting nodes to float32, that we need to carefully modify in order to exactly replicate the accuracy of the original model.

- Link to supported operators: https://github.com/onnx/onnx/blob/main/docs/Operators.md

#### <ins>ColumnTransformer to separate inputs in scikit-learn</ins>

When converting a scikit-learn model to ONNX format, it is possible to use a ColumnTransformer to separate the inputs into multiple columns and apply different transformations to each column. Please refer to [Remapping model inputs](#2-1) section for implementation details.


#### <ins>Different input dimensions</ins>

When defining a dynamic input shape for an ONNX model, you can use strings as dimension names for the batch dimension (the first dimension of the input shape). By convention, the two most common strings used for batch dimensions are 'batch_size' and 'N'. However, any string can be used as long as it is unique and consistent across all input tensors. We can use 'None' to represent an undefined batch size.

**If the dimensions are different across inputs you need to define different strings to define the inputs.**

    Example:

```python
from skl2onnx import convert_sklearn
input_shape_1 = (None, 28, 28, 1)  # batch size is undefined
input_shape_2 = (10, 32)  # batch size is 10

initial_types = [('input_with_batchsize_defined', FloatTensorType(input_shape_1)),
                ('input_with_undefined_batchsize', FloatTensorType(input_shape_2))]

onnx_model = convert_sklearn(model, initial_types=initial_types)
```

#### <ins>Sparse to dense for CountVectorizer as input to XGBoost</ins>
When using CountVectorizer to encode text data for use with XGBoost, the resulting feature matrix is typically sparse. When converting a sparse CountVectorizer feature matrix to ONNX format for use with XGBoost, it is important to ensure that the resulting ONNX model can handle sparse inputs efficiently.

In the following example, we first fit a CountVectorizer on the training data to obtain a sparse feature matrix, X_train. We then convert this matrix to a dense format. The resulting dense feature matrix is then used as input to an ONNX model that represents an XGBoost classifier.


In [ ]:
import onnx
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer
from skl2onnx.common.data_types import StringTensorType
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from onnxmltools.convert import convert_xgboost

from acslibrary.onnx_utils.conversion import sklearn_to_onnx

# Load the 20 Newsgroups dataset
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(newsgroups.data, newsgroups.target, test_size=0.2, random_state=42)

# Fit a CountVectorizer on the training data
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
initial_types=[("input", StringTensorType([1,None]))]
vector_onnx_model = sklearn_to_onnx(vectorizer, initial_types=initial_types, save_onnx_filename="./models/vector.onnx")

# Convert the sparse feature matrix to a dense format
X_train_dense = X_train.todense()

xgb=XGBClassifier()
xgb.fit(X_train_dense,y_train)
input = [onnx.helper.make_tensor_value_info('input', onnx.TensorProto.FLOAT, X_train_dense.shape)]
xgb_onnx_model = convert_xgboost(xgb, initial_types=input)
onnx.save(xgb_onnx_model, './models/xgb.onnx')

combined_model = onnx.compose.merge_models(
    vector_onnx_model, xgb_onnx_model,
    io_map=[("variable", "input")]
)
onnx.save(combined_model, './models/combined_model.onnx')


## Graph modifications <a class="anchor" id="2"></a>

### OnnxRuntime Optimization <a class="anchor" id="2-0"></a>

ONNX Runtime defines the GraphOptimizationLevel enum to determine which of the aforementioned optimization levels will be enabled. Choosing a level enables the optimizations of that level, as well as the optimizations of all preceding levels. For example, enabling Extended optimizations, also enables Basic optimizations. The mapping of these levels to the enum is as follows:

- GraphOptimizationLevel::ORT_DISABLE_ALL -> Disables all optimizations
- GraphOptimizationLevel::ORT_ENABLE_BASIC -> Enables basic optimizations
- GraphOptimizationLevel::ORT_ENABLE_EXTENDED -> Enables basic and extended optimizations
- GraphOptimizationLevel::ORT_ENABLE_ALL -> Enables all available optimizations including layout optimizations

Aditional Resources:
* https://fs-eire.github.io/onnxruntime/docs/performance/graph-optimizations.html

In [ ]:
from acslibrary.onnx_utils.graph_modifications import optimize_graph

input_model_filename = "./models/resampler.onnx"
optimized_model_filename = "./models/resampler_optimized.onnx"
optimization_level = "ORT_ENABLE_EXTENDED"
optimize_graph(input_model_filename, optimized_model_filename, optimization_level)

### Remapping model inputs <a class="anchor" id="2-1"></a>

The `ColumnTransformer` is a scikit-learn transformer that applies different transformers to different columns of the input data. This can be useful when working with datasets that contain both numerical and categorical features, for example. The ColumnTransformer allows you to apply different transformations to each feature type, and then combine the results into a single output.

The function `acslibrary.onnx_utils.graph_modifications.add_feature_concat` inserts the ColumnTransformer at the beginning of an already existing algorithm or pipeline. This way, when an model is converted to onnx, the inputs can be passed separated and the model compacts them into a single array.

This function accepts multiple inputs:
* Option 1: Pass the desired algorithm to the function, with or without algorithm argumentes
* Option 2: Run the function over an existing sklearn Pipeline

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

from acslibrary.onnx_utils.graph_modifications import add_feature_concat

# Imagine that we have a dataframe with the following columns
col_names = ['var1', 'var2', 'var3', 'var4']

print ("Option 1: pass the desired algorithm to the function")
model_op1 = add_feature_concat(col_names, algorithm=KNeighborsRegressor(), algorithm_name='knn')
print(model_op1)

# We can add the parameters that we want to the algorithm, and we can also not pass a name
print ("Option 1: pass the desired algorithm with parameters and without the name")
model_op1 = add_feature_concat(col_names, algorithm=KNeighborsRegressor(n_neighbors=10, algorithm='ball_tree', leaf_size=50))
print(model_op1)

In [ ]:
# We can also pass an already build pipeline and the column transformer will be added at the beginning
print ("Option2: Pass an already built pipeline ")
model_op2 = Pipeline([('pca', PCA(n_components=2)), ('knn', KNeighborsRegressor())])
print('Model before input feature separation')
print(model_op2)

add_feature_concat(col_names, pipeline=model_op2)
print('Model after input feature separation')
print(model_op2)

### Renaming model outputs <a class="anchor" id="2-2"></a>

The output of existing ONNX model can be changed from standard [0, 1, 2, ...] to custom defined values: either a new list of int, strings or float, i.e ["A", "B", "C", ...].

The mapping from existing output values to new output values is performed by modifying the graph. The existing label output node (or every output node) is removed and substituted it by a LabelEncoder node.

`rename_onnx_outputs` allows deciding if removing:
* all output nodes: "all"
* specific output node: i.e. "label"

In [ ]:
from acslibrary.onnx_utils.graph_modifications import rename_onnx_outputs

original_model_path = './models/GT_clustering.onnx'

input_mapping = [0, 1, 2]
mapping_examples = {
    'int': [[1, 2, 3], int],
    'float': [[1.0, 2.0, 3.0], float],
    'str': [["A", "B", "C"], str]
}

for new_output_type, output_mapping in mapping_examples.items():
    new_model_path = f'./models/GT_clustering_new_output_{new_output_type}.onnx'

    rename_onnx_outputs(original_model_path, input_mapping, output_mapping[0],
                                new_model_path, output_node_to_replace="all")

### Merging graphs <a class="anchor" id="2-3"></a>

In [ ]:
import onnx
import onnx.utils
from onnx.version_converter import convert_version

# Load the two ONNX models
model1 = onnx.load(model1_path)
model2 = onnx.load(model2_path)

# Check operator set versions of the two models
print("Model 1 operator set version:", model1.opset_import[0].version)
print("Model 2 operator set version:", model2.opset_import[0].version)

# Set the target operator set version
target_opset_version = 16

# Update the first model to the target operator set version
if model1.opset_import[0].version < target_opset_version:
    model1 = convert_version(model1, target_version=target_opset_version)

# Merge the two models
merged_model = onnx.compose.merge_models(model1, model2,
                                         io_map=[('model1_output','model2_input')])

# Save the merged model
merged_model_path = './models/merged_model.onnx'
onnx.save(merged_model, merged_model_path)

# Check operator set version of the merged model
print("Merged model operator set version:", merged_model.opset_import[0].version)

## Time Series Data <a class="anchor" id="3-2"></a>


### KShape to ONNX <a class="anchor" id="3-1"></a>

*k-Shape* is a highly accurate and efficient unsupervised method for univariate and multivariate time-series clustering. 

The simplest way to use it in python is with the _tslearn.clustering.KShape_ library; however, the resulting model cannot be directly converted to onnx. KShape algorithm has been transformed to a pytorch model to be compatible with onnx transformation. Moreover, onnx conversion does not support float64 transformation in every node, therefore the resulting model needs to be modified in order to remove/add the corresponding nodes in float64 type.

The pipeline to obtain a k-shape onnx model for inference follows:

    1. Train original tslearn.clustering.KShape model.
    2. Use tslearn k-shape cluster centers to fit Pytorch KShape (Aizon custom)
    3. Convert Pytorch k-shape model to onnx
    4. Modify model from step 3 to ensure all nodes support float64 types

In [ ]:
import numpy as np
import torch

from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.clustering import KShape as tslearn_KShape

import onnxruntime as ort
from acslibrary.onnx_utils.kshape import kshape_to_onnx


# Example data generation and split into train/test
seed = 0
np.random.seed(seed)
X_train, y_train, X_test, y_test = CachedDatasets().load_dataset("Trace")
# Keep first 3 classes and 50 first time series
X_train = X_train[y_train < 4]
X_train = X_train[:50]
np.random.shuffle(X_train)
X_train=torch.from_numpy(X_train)

X_train = TimeSeriesScalerMeanVariance().fit_transform(X_train)

onnx_save_file = "./models/kshape.onnx"
print("Exporting kshape model to onnx format")
kshape_to_onnx(X_train, num_clusters=3,seed=seed, save_onnx_filename=onnx_save_file)
sess_kshape = ort.InferenceSession(onnx_save_file)
onnx_pred = sess_kshape.run(None,{"input_1": X_test[:20],})
print(f"ONNX model saved in {onnx_save_file}")

print("Fitting training data to original tslearn kshape for comparison")
ks = tslearn_KShape(n_clusters=3, verbose=True, random_state=seed)
tslearn_model = ks.fit(X_train)
sklearn_pred = tslearn_model.predict(X_test[:20])

## Compare
print(f'\nONNX results: \n{onnx_pred[0]}')
print(f'\nTslearn results: \n{sklearn_pred}')

### DTW to ONNX <a class="anchor" id="3-2"></a>

DTW is used as a distance metric in combination with a clustering algorhtm (i.e. k-means). It was converted to ONNX but it is still very slow - however the converted model is documented hereby for future use or development.

In [ ]:
import onnxruntime as ort
import torch
import onnx

from acslibrary.onnx_utils.dtw import dtw_to_onnx
from acslibrary.onnx_utils.graph_modifications import optimize_graph


dummy_input_one = torch.Tensor([1, 2, 3, 10, 8, 5, 4, 1, 2])
dummy_input_two = torch.Tensor([4, 5, 6, 2, 2])

input_data = {
        'input_data_1': dummy_input_one.numpy(),
        'input_data_2': dummy_input_two.numpy(),
    }

filename = './models/dtw_torch.onnx'
print("Exporting DTW model to onnx format")
dtw_to_onnx(filename)

# optimize onnx graph
filename_optimized = './models/dtw_optimized.onnx'
optimize_graph(filename, optimized_model_filename=filename_optimized)
so = ort.SessionOptions()
sess_dtw = ort.InferenceSession(onnx.load(filename_optimized).SerializeToString(), so, providers=['CPUExecutionProvider'])
onnx_pred = sess_dtw.run(None, input_data)

print(f'\nONNX results: \n{onnx_pred[0]}')

### Resampling <a class="anchor" id="3-2"></a>

When loading time series data many times there exists redundancy and we want to compress the information across time for every batch.

In order to reduce this redundancy, the function `acslibrary.ETL.scada_data.load_data` applies custom defined resampling, consisting in averaging the values of time series data within batches. In some cases, it is necessary to have additional information to describe how data has changed over time. This information can be extracted either with the difference of values between consecutive data points (diff) or the percentage change between the current and a prior data points (perc_change).

The above mentioned function has been converted to ONNX format by defining both regular Resampler and Resampler with derivatives as Pytorch nn modules, to be merged before models when we want to resample input data.

These onnx models can be trained and merged as follows:

#### Train Resampler onnx model (with or without derivatives)

In [ ]:
import torch
from acslibrary.onnx_utils import resampling

#Generate dummy input to feed model to export
dummy_input = torch.randn(10,60).type(torch.float64)

# Generate Resampler onnx model
resampler_filename="./models/resampler.onnx"
resampling.resampler_to_onnx(dummy_input, n_chunks=9, save_onnx_filename=resampler_filename)

# Generate Resampler onnx model with derivative
resampler_derivative_filename="./models/resampler_with_derivative.onnx"
resampling.resampler_to_onnx(dummy_input, n_chunks=9, save_onnx_filename=resampler_derivative_filename, derivative=True)

#### Example: Use Resampler onnx model instead of `load_data`

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from skl2onnx.common.data_types import FloatTensorType

import onnx
import onnx.utils
from onnx.version_converter import convert_version

from acslibrary.onnx_utils.conversion import sklearn_to_onnx


# Load Resampler ONNX model
resampler_derivative_filename="./models/resampler_with_derivative.onnx"
resample_diff_model = onnx.load(resampler_derivative_filename)

# Convert custom model to ONNX, i.e Kmeans 
X = np.arange(20).reshape(10, 2)
kmeans_model = KMeans(n_clusters=2)
kmeans_model.fit(X)
kmean_onnx_filename = './models/k_means.onnx'
sklearn_to_onnx(kmeans_model, initial_types=[('kmeans_input', FloatTensorType([1, 34]))], 
                save_onnx_filename=kmean_onnx_filename, target_opset=16)
# Update the model to use the target operator set version
k_means_onnx = onnx.load(kmean_onnx_filename)

# Update the kmeans model to the target operator set version of the resampler model
k_means_onnx = convert_version(k_means_onnx,target_version=16)

#Generate combined model
resample_k_means_model = onnx.compose.merge_models(resample_diff_model, k_means_onnx, 
                                                   io_map=[('resampler_output','kmeans_input')],
                                              )

onnx.checker.check_model(resample_k_means_model)
onnx.save(resample_k_means_model, './models/res_diff_kmeans.onnx')

## ONNX Prediction <a class="anchor" id="4"></a>

In order to predict the output of a given ONNX model, we need to create an onnxruntime inference session and then run it passing the onnx model and a given set of input values.

To ease this process, one can use the onnx_predict method as in the following example. The function accepts either a loaded onnx model or the path of an existing onnx model. It returns the inference results in a dictionary with the corresponding output labels from the graph. If the format of the inputs is incorrect, it will display an error message indicating the correct shape or type of the data to be passed.


In [ ]:
import numpy as np

from acslibrary.onnx_utils.conversion import onnx_predict

incorrect_vals = np.array([[1,2]], dtype=np.float32)
output = onnx_predict(incorrect_vals, onnx_model_path="./models/sklearn_tree_model.onnx")
print("Result",output)

print("____________")
corrected_vals = [np.array([[1,2]], dtype=np.float32)]
output = onnx_predict(corrected_vals, onnx_model_path="./models/sklearn_tree_model.onnx")
print("Result",output)

## SHAP values <a class="anchor" id="5"></a>


**Shapley Additive Explanations (SHAP)** values are a popular method for interpreting and explaining the predictions made by machine learning models. They provide insights into the contribution of each feature or variable towards a particular prediction. Shap values quantify the marginal contribution of each feature to the prediction when considering different possible combinations of features. By calculating Shap values, we can understand how individual features influence the model's output and gain valuable insights into the decision-making process.

To further enhance the interpretability and applicability of machine learning models, the function `acslibrary.onnx_utils.shap.concat_with_shap` has been developed to seamlessly integrate SHAP values with an existing ONNX model. This function allows us to concatenate the SHAP value calculation module with the ONNX model, providing a streamlined approach to transforming SHAP values into the ONNX format.

The function leverages a kernel explainer, which is a widely used method for estimating SHAP values. Kernel explainer approximates SHAP values by creating a reference dataset and sampling from it to evaluate the model's behavior across different feature combinations. By incorporating this explainer, we can effectively compute SHAP values for the given model and utilize them in conjunction with the ONNX representation.

**Limitations:** the training and inference time required to run the model with shap increases exponentially with the number of input variables. Take this limitation into account when converting models to be used in the platform. To ensure inference time below 1 minute (platform limitation for inference time), the maximum amount of variables is 10.

#### Example: Concatenate shap to an existing onnx model

In [ ]:
import pandas as pd
import onnx

from acslibrary.onnx_utils.shap import concat_with_shap

# Load training data used to fit base model
kwargs = {'sep': '\t', 'header': 'infer', 'encoding': 'utf-8', 'index_col': [0]}
train_data_file = "./models/training_data_X.csv"
X_train = pd.read_csv(train_data_file, index_col=0).values

# Load base model in onnx format:
# this model needs to have inputs concatenated into a single input
# i.e. do not use "feature_concat" function
base_model_path = "./models/Frac1_pls.onnx"
onnx_base_model = onnx.load(base_model_path)

# Define list with names for separate inputs of the final model
input_names = ['input_33', 'input_47', 'input_6']
# Define path to save final model
save_onnx_filename = "./models/Frac1_pls_with_shap.onnx"

# Create final model concatenating shapley values to base model
shap_onxx_model = concat_with_shap(onnx_base_model, X_train, input_names, save_onnx_filename)

Run inference to obtain shap values with the resulting model

In [ ]:
import numpy as np
import onnxruntime as ort

# Run inference with the model with shap values
sess_options = ort.SessionOptions()
sess_options.log_severity_level = 3  # hide warning messages
sess = ort.InferenceSession(shap_onxx_model.SerializeToString(),
                                    sess_options=sess_options)

inference_data = {
    k.name: [[v]] for k, v in zip(sess.get_inputs(), np.array(X_train[2]))
}
results_onnx = sess.run(None, inference_data)

results_dict = {}
results_dict["prediction"] = results_onnx[0][0].tolist()
results_dict["shap_values"] = results_onnx[1][-1].tolist()
results_dict["expected_score"] = float(results_onnx[2])
print(results_dict)